Student Name: Rickell McDowall

Student ID: 101073978


In [ ]:
import sqlite3
from datetime import datetime

class Expense:
    def __init__(self, category, amount, date=None):
        self.category = category
        self.amount = amount
        self.date = date if date else datetime.now().strftime('%Y-%m-%d')

class ExpenseTracker:
    def __init__(self):
        self.conn = sqlite3.connect('track_expense.db')
        self.cursor = self.conn.cursor()
        self.create_table()
        self.budget_limits = {}

    def create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS expenses (
                id INTEGER PRIMARY KEY,
                category TEXT,
                amount REAL,
                date TEXT
            )
        ''')
        self.conn.commit()

    def close(self):
        self.conn.close()

    def add_expense(self, expense):
        try:
            self.cursor.execute('''
                INSERT INTO expenses (category, amount, date) VALUES (?, ?, ?)
            ''', (expense.category, expense.amount, expense.date))
            self.conn.commit()  # Commit the transaction
            print("Expense added successfully.")
        except Exception as e:
            print("Error adding expense:", e)

    def reset_expenses(self):
      try:
          self.cursor.execute('SELECT id, category, amount, date FROM expenses')
          expenses = self.cursor.fetchall()
          print("ID\tCategory\tAmount\tDate")
          print("----------------------------------------------")
          for expense in expenses:
              print(f"{expense[0]}\t{expense[1]}\t${expense[2]}\t{expense[3]}")

          expense_id = input("Enter the ID of the expense you want to reset (or press Enter to cancel): ")
          if not expense_id:
              print("Operation canceled.")
              return

          self.cursor.execute('DELETE FROM expenses WHERE id = ?', (expense_id,))
          self.conn.commit()
          print("Expense reset successfully.")
      except Exception as e:
        print("Error resetting expense:", e)

    def set_budget_limit(self, category, limit):
        self.budget_limits[category] = limit
        print(f"Budget limit for {category} set to ${limit}.")

    def get_monthly_expense_report(self, month, year):
        try:
            start_date = f'{year}-{month:02d}-01'
            end_date = f'{year}-{month:02d}-31'
            self.cursor.execute('''
                SELECT category, SUM(amount) FROM expenses
                WHERE date BETWEEN ? AND ?
                GROUP BY category
            ''', (start_date, end_date))
            monthly_expenses = self.cursor.fetchall()

            total_monthly_expense = sum(amount for _, amount in monthly_expenses)

            report = f"Monthly Expense Report for {month}/{year}\n"
            report += "--------------------------------------------\n"
            report += f"Total Monthly Expense: ${total_monthly_expense}\n\n"
            report += "Category\t\tExpense\n"
            report += "--------------------------------------------\n"
            for category, amount in monthly_expenses:
                report += f"{category}\t\t${amount}\n"
            return report
        except Exception as e:
            print("Error generating monthly expense report:", e)
            return ""

    def get_weekly_expense_report(self, start_date, end_date):
        try:
            self.cursor.execute('''
                SELECT category, SUM(amount) FROM expenses
                WHERE date BETWEEN ? AND ?
                GROUP BY category
            ''', (start_date, end_date))
            weekly_expenses = self.cursor.fetchall()

            total_weekly_expense = sum(amount for _, amount in weekly_expenses)

            report = f"Weekly Expense Report for {start_date} to {end_date}\n"
            report += "--------------------------------------------\n"
            report += f"Total Weekly Expense: ${total_weekly_expense}\n\n"
            report += "Category\t\tExpense\n"
            report += "--------------------------------------------\n"
            for category, amount in weekly_expenses:
                report += f"{category}\t\t${amount}\n"
            return report
        except Exception as e:
            print("Error generating weekly expense report:", e)
            return ""

def print_menu():
    print("\nExpense Tracker Menu:")
    print("1. Add Expense")
    print("2. Generate Monthly Expense Report")
    print("3. Generate Weekly Expense Report")
    print("4. Reset Expenses")
    print("5. Set Budget Limit")
    print("6. Exit")

def print_add_expense_menu():
    print("\nAdd Expense Menu:")
    print("1. Add Expense")
    print("q. Go back to Main Menu")

def print_monthly_report_menu():
    print("\nMonthly Report Menu:")
    print("1. Generate Monthly Expense Report")
    print("q. Go back to Main Menu")

def print_weekly_report_menu():
    print("\nWeekly Report Menu:")
    print("1. Generate Weekly Expense Report")
    print("q. Go back to Main Menu")

def print_set_budget_menu():
    print("\nSet Budget Limit Menu:")
    print("1. Set Budget Limit")
    print("q. Go back to Main Menu")

if __name__ == "__main__":
    tracker = ExpenseTracker()

    while True:
          print_menu()
          choice = input("Enter your choice: ")

          if choice == '1':
              while True:
                  print_add_expense_menu()
                  add_choice = input("Enter your choice: ")
                  if add_choice == '1':
                      category = input("Enter expense category: ")
                      amount = float(input("Enter expense amount: "))
                      date = input("Enter expense date (YYYY-MM-DD) or press Enter for today's date: ")
                      if not date:
                          date = None
                      tracker.add_expense(Expense(category, amount, date))
                  elif add_choice.lower() == 'q':
                      break
                  else:
                      print("Invalid choice. Please enter 1 or 'q'.")
          elif choice == '2':
              while True:
                  print_monthly_report_menu()
                  monthly_choice = input("Enter your choice: ")
                  if monthly_choice == '1':
                      month = None
                      while month is None or month < 1 or month > 12:
                          try:
                              month = int(input("Enter month (1-12): "))
                              if month < 1 or month > 12:
                                  print("Invalid month. Please enter a number between 1 and 12.")
                          except ValueError:
                              print("Invalid input. Please enter a number.")
                      year = int(input("Enter year: "))
                      print(tracker.get_monthly_expense_report(month, year))
                  elif monthly_choice.lower() == 'q':
                      break
                  else:
                      print("Invalid choice. Please enter 1 or 'q'.")
          elif choice == '3':
              while True:
                  print_weekly_report_menu()
                  weekly_choice = input("Enter your choice: ")
                  if weekly_choice == '1':
                      start_date = input("Enter start date of the week (YYYY-MM-DD): ")
                      end_date = input("Enter end date of the week (YYYY-MM-DD): ")
                      print(tracker.get_weekly_expense_report(start_date, end_date))
                  elif weekly_choice.lower() == 'q':
                      break
                  else:
                      print("Invalid choice. Please enter 1 or 'q'.")
          elif choice == '4':
              tracker.reset_expenses()
          elif choice == '5':
              while True:
                  print_set_budget_menu()
                  budget_choice = input("Enter your choice: ")
                  if budget_choice == '1':
                      category = input("Enter expense category: ")
                      limit = float(input("Enter budget limit for this category: "))
                      tracker.set_budget_limit(category, limit)
                  elif budget_choice.lower() == 'q':
                      break
                  else:
                      print("Invalid choice. Please enter 1 or 'q'.")
          elif choice == '6':
              print("Exiting...")
              tracker.close()
              break
          else:
              print("Invalid choice. Please enter a number between 1 and 6.")



tracker.close()




Expense Tracker Menu:
1. Add Expense
2. Generate Monthly Expense Report
3. Generate Weekly Expense Report
4. Reset Expenses
5. Set Budget Limit
6. Exit
Enter your choice: 1

Add Expense Menu:
1. Add Expense
q. Go back to Main Menu
Enter your choice: 1
Enter expense category: Clothing
Enter expense amount: 100
Enter expense date (YYYY-MM-DD) or press Enter for today's date: 2024-02-09
Expense added successfully.

Add Expense Menu:
1. Add Expense
q. Go back to Main Menu
Enter your choice: q

Expense Tracker Menu:
1. Add Expense
2. Generate Monthly Expense Report
3. Generate Weekly Expense Report
4. Reset Expenses
5. Set Budget Limit
6. Exit
Enter your choice: 2

Monthly Report Menu:
1. Generate Monthly Expense Report
q. Go back to Main Menu
Enter your choice: 1
Enter month (1-12): 1
Enter year: 2024
Monthly Expense Report for 1/2024
--------------------------------------------
Total Monthly Expense: $335.0

Category		Expense
--------------------------------------------
Car		$90.0
Clothi

KeyboardInterrupt: Interrupted by user